In [1]:
import baseline_methods as bm
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2

In [2]:
#cn_delay = np.load('./cdata/China.npz')
#cn_delay = cn_delay['data']
#cn_delay = cn_delay.transpose(1,0,2)
cn_delay = np.load('./cdata/delay.npy')
print(cn_delay.shape)
# Shape(N, T, F)

#cn_delay_1 arrival delay 
cn_delay_1 = cn_delay[:,:,0]
#cn_delay_1 departure delay 
cn_delay_2 = cn_delay[:,:,1]

(50, 27468, 2)


In [3]:
cn_delay_1.shape

(50, 27468)

# HA

In [6]:
y_predict,y_test = bm.historical_average_predict(cn_delay_1,period=2*18,test_ratio=0.2)
ha_a_mae,ha_a_rmse,ha_a_mape =bm.test_error(y_predict,y_test)
y_predict,y_test = bm.historical_average_predict(cn_delay_2,period=2*18,test_ratio=0.2)
ha_d_mae,ha_d_rmse,ha_d_mape = bm.test_error(y_predict,y_test)
print(f"The test error for arrival prediction delay using HA is {round(ha_a_mae,3), round(ha_a_rmse,3), round(ha_a_mape,3)}")
print(f"The test error for delay prediction delay using HA is {round(ha_d_mae,3), round(ha_d_rmse,3), round(ha_d_mape,3)}")

The test error for arrival prediction delay using HA is (10.72, 13.123, 0.889)
The test error for delay prediction delay using HA is (10.441, 12.929, 0.807)


# VAR

In [8]:
time = [2,5,11]

In [8]:
y_predict_a,y_test_a = bm.var_predict(cn_delay_1,test_ratio=0.2)
y_predict_d,y_test_d = bm.var_predict(cn_delay_2,test_ratio=0.2)
var_a_mae = []
var_a_rmse = []
var_a_mape = []
var_d_mae = []
var_d_rmse = []
var_d_mape = []
for i in time:
    a,b,c =bm.test_error(y_predict_a[i,:,:].T,y_test_a)
    var_a_mae.append(round(a,3))
    var_a_rmse.append(round(b,3))
    var_a_mape.append(round(c,3))
    a,b,c =bm.test_error(y_predict_d[i,:,:].T,y_test_d)
    var_d_mae.append(round(a,3))
    var_d_rmse.append(round(b,3))
    var_d_mape.append(round(c,3))

for i, x in enumerate(zip(var_a_mae, var_a_rmse, var_a_mape)):
    print(f"Error of VAR in {time[i]+1}-step arrival: {x}")

for i, x in enumerate(zip(var_d_mae, var_d_rmse, var_d_mape)):
    print(f"Error of VAR in {time[i]+1}-step departure: {x}")

Error of VAR in 3-step arrival: (9.3, 11.621, 0.873)
Error of VAR in 6-step arrival: (9.819, 12.136, 0.845)
Error of VAR in 12-step arrival: (10.571, 12.848, 0.806)
Error of VAR in 3-step departure: (9.718, 12.065, 0.784)
Error of VAR in 6-step departure: (10.324, 12.635, 0.734)
Error of VAR in 12-step departure: (10.918, 13.16, 0.682)


# ARIMA

In [5]:
y_predict_a, y_test_a=bm.arima_predict(cn_delay_1,test_ratio=0.02)
y_predict_d, y_test_d=bm.arima_predict(cn_delay_2,test_ratio=0.02)

In [6]:
print(np.sum(np.isnan(y_predict_a)))
print(y_predict_a.shape)
print(y_test_a.shape)

0
(12, 549, 50)
(50, 549)


In [9]:
arima_a_mae = []
arima_a_rmse = []
arima_a_mape = []
arima_d_mae = []
arima_d_rmse = []
arima_d_mape = []
for i in time:
    a,b,c =bm.test_error(y_predict_a[i,:,:].T,y_test_a)
    arima_a_mae.append(round(a,3))
    arima_a_rmse.append(round(b,3))
    arima_a_mape.append(round(c,3))
    a,b,c =bm.test_error(y_predict_d[i,:,:].T,y_test_d)
    arima_d_mae.append(round(a,3))
    arima_d_rmse.append(round(b,3))
    arima_d_mape.append(round(c,3))

for i, x in enumerate(zip(arima_a_mae, arima_a_rmse, arima_a_mape)):
    print(f"Error of ARIMA in {time[i]+1}-step arrival: {x}")

for i, x in enumerate(zip(arima_d_mae, arima_d_rmse, arima_d_mape)):
    print(f"Error of ARIMA in {time[i]+1}-step departure: {x}")

Error of ARIMA in 3-step arrival: (11.085, 13.239, 0.781)
Error of ARIMA in 6-step arrival: (11.032, 13.185, 0.785)
Error of ARIMA in 12-step arrival: (11.099, 13.263, 0.779)
Error of ARIMA in 3-step departure: (11.61, 13.66, 0.664)
Error of ARIMA in 6-step departure: (11.593, 13.658, 0.651)
Error of ARIMA in 12-step departure: (11.639, 13.687, 0.657)


# SVM

In [4]:
y_predict_a, y_test_a=bm.var_predict_svr(cn_delay_1,test_ratio=0.2)
y_predict_d, y_test_d=bm.var_predict_svr(cn_delay_2,test_ratio=0.2)
np.savez("tmp_svr.npz", y_predict_a=y_predict_a, y_test_a=y_test_a, y_predict_d=y_predict_d, y_test_d=y_test_d)

100%|██████████| 50/50 [2:12:49<00:00, 159.39s/it]  


result (50, 5446, 12); df_test (50, 5458)


100%|██████████| 50/50 [1:57:26<00:00, 140.94s/it]  

result (50, 5446, 12); df_test (50, 5458)


In [7]:
#y_test_a = y_test_a[:-36]
time = [2, 5, 11]
out_len = 12
var_a_mae = []
var_a_rmse = []
var_a_mape = []
var_d_mae = []
var_d_rmse = []
var_d_mape = []

for i in time:
    shift = i+1
    num_sample = y_test_a.shape[-1]
    a,b,c =bm.test_error(y_predict_a[:, :, i],y_test_a[:, shift:num_sample-(out_len-shift)])
    # y_predict_a: [T, N]  y_test_a: [T, N], shift `i+1` slots
    var_a_mae.append(round(a,3))
    var_a_rmse.append(round(b,3))
    var_a_mape.append(round(c,3))
    a,b,c =bm.test_error(y_predict_d[:, :, i],y_test_d[:, shift:num_sample-(out_len-shift)])
    # y_predict_a: [T, N]  y_test_a: [T, N], shift `i+1` slots
    var_d_mae.append(round(a,3))
    var_d_rmse.append(round(b,3))
    var_d_mape.append(round(c,3))

for i, x in enumerate(zip(var_a_mae, var_a_rmse, var_a_mape)):
    print(f"Error of SVM in {(time[i]+1)}-step arrival: {x}")

for i, x in enumerate(zip(var_d_mae, var_d_rmse, var_d_mape)):
    print(f"Error of SVM in {(time[i]+1)}-step departure: {x}")

Error of SVM in 3-step arrival: (9.904, 12.457, 0.969)
Error of SVM in 6-step arrival: (10.315, 12.871, 1.001)
Error of SVM in 12-step arrival: (10.528, 13.113, 1.02)
Error of SVM in 3-step departure: (10.185, 12.805, 0.899)
Error of SVM in 6-step departure: (10.474, 13.128, 0.923)
Error of SVM in 12-step departure: (10.62, 13.308, 0.937)
